Training auf verschlüsselten Daten
===
Zuerst werden für das Training alle Daten geladen und notwendigen Funktion definiert. 

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np
import tenseal as ts
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

def fit(x, y):
	regression = np.c_[x, np.ones(len(x))]

	weights = np.ones(regression.shape[1])

	norma = 1
	learning_rate = 0.00001
	epsilon = 0.9
	while(norma > epsilon):
		y_pred = regression @ weights.T
		dw = regression.T @ (y - y_pred)
		norma = np.sum(np.sqrt(np.square(dw)))

		weights = weights.T + (learning_rate * dw)
	return weights

def predict(w, x):
	return w[:-1] @ (np.array(x).T) + w[-1]

weights = fit(X_train, y_train)

def meanError(y_test, pred):
	print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
	print('Mean Squared Error:', mean_squared_error(y_test, pred))
	print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

Für das Training auf verschlüsselten Daten müssen auch die Gewichte der Linearer Regression verschlüsselt werden. Nachdem Training können die Gewichte wieder entschlüsselt und wieder für die Inferenz genutzt werden. Hierfür verschlüsselt der Datenbesitzer seine Daten und verschickt diese zusammen mit dem öffentlichen Schlüssel.

In [2]:
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

def encryptX(data, context):
    return [[ts.ckks_vector(context, [feature]) for feature in item] for item in data]

def encryptY(data, context):
    return [ts.ckks_vector(context, [label]) for label in data]

def decrypt(data):
    return [item.decrypt() for item in data]

seal_x_train = encryptX(X_train,ctx_training)
seal_y_train = encryptY(y_train,ctx_training)

Um das Netz zu trainieren gibt es zwei Möglichkeiten:
* Nach jedem Trainingsschritt können die Gewichte entschlüsselt werden, durch den Besitzer des zugehörigen privaten Schlüssels, um dann wieder verschlüsselt zu werden. Dies würde erlauben die Berechnung so oft wie gewünscht durchzuführen. Das Problem hierbei ist, dass dies ein großer Zeit und Rechenaufwand mit sich bringt.
* Die Gewichte müssen aber nicht nach jedem Trainingsschritt entschlüsselt werden und es kann für eine begrenzte Anzahl von Operationen trainiert werden. Diese Variante verringert aber die erreichbare Genauigkeit der trainierten Gewichte.
* Als letzte Option, welche auch zu kein Ausstausch mit dem Datenbesitzer fällt ist das Bootstrapping, da damit unbegrenzt Berechnungen durchgeführt werden können

Kein Ausstausch mit Datenbesitzer
---
Um den Aufwand für den Datenbesitzer niedrig zu halten wird die zweite Variante implementiert und getestet. Hierfür wird die fit Funktion verändert, da nicht alle Vorgänge durchführbar sind mit verschlüsselten Daten. So können zum Beispiel die Gewichte nicht solange optimiert werden bis diese konvergieren, weshalb es nur wenige Trainingsphasen geben wird. Damit das Training trotzdem erfolgreich ist ist es hielfreich verschiedene Lernraten auszuprobieren, um die Gewichte gut zu approximieren. Diese Variante ist also pures Leveled HE.

In [3]:
hyperparameter = [2, 1, 0.1, 0.01, 0.007, 0.003, 0.002, 0.001, 0.0009, 0.0007, 0.0003, 0.0001, 0.00001]

def fit(x, y, context):
	regression = np.c_[x, np.ones(len(x))]

	weights = np.ones(regression.shape[1])
	e_weights = np.array(encryptY(weights, context))

	all_weights = []
	for learning_rate in hyperparameter:
		hyper_weights = e_weights
		for i in range(2):
			y_pred = regression @ hyper_weights.T
			dw = regression.T @ (y - y_pred)
			hyper_weights = hyper_weights.T + (learning_rate * dw)

		all_weights.append((learning_rate, hyper_weights))

	return all_weights

Das Training gibt also eine Liste an verschiedenen Gewichten aus für die unterschiedlichen Lernraten.

In [4]:
all_weights = fit(seal_x_train, seal_y_train, ctx_training)

e_weights = [[e_value.decrypt()[0] for e_value in e_tuple[1]] for e_tuple in all_weights]

Die verschiedenen Versionen der Gewichte können nun einzelnen geprüft werden auf dem Testdatensatz. Dadurch kann bestimmt werden welche Lernrate für diese Aufgabe die Beste ist.

In [5]:
min = []
for idx, learning_rate in enumerate(hyperparameter):
    result = predict(e_weights[idx], X_test)
    min.append(np.sqrt(mean_squared_error(y_test, result)))
best_weight_idx = np.argmin(min)
print(f"Beste Gewichte sind die mit der Learning rate von {hyperparameter[best_weight_idx]}")

Beste Gewichte sind die mit der Learning rate von 0.002


Die besten trainierten Gewichte haben eine höhere Abweichung als die Gewichte der unverschlüsselten Modelle. Es kommt aber nahe dran und kann noch weiter verbessert werden mit mehr Trainingsiterationen.

In [6]:
from random import randint

index = randint(0, len(X_test)-1)
print(f"""
Eingabe: {X_test[index]}
Wahres Ergebnis: {y_test[index]}
Ergebnis aus dem verschlüsselt trainierten Modell: {predict(e_weights[best_weight_idx], X_test[index])}
Ergebnis aus dem nicht verschlüsselt trainierten Modell: {predict(weights, X_test[index])}
""")


Eingabe: [ 1.64384411 -0.13197948  1.16062026  0.52740629]
Wahres Ergebnis: 2
Ergebnis aus dem verschlüsselt trainierten Modell: 1.5720731776464807
Ergebnis aus dem nicht verschlüsselt trainierten Modell: 1.5767759592337507



In [7]:
result = predict(e_weights[best_weight_idx], X_test)
meanError(y_test, result)

Mean Absolute Error: 0.3158160197306934
Mean Squared Error: 0.13016683310297522
Mean Root Squared Error: 0.36078640925480443


Der Unterschied im einzelnen Beispiel ist minimal und auch die eizelnen Metriken zeigen auf, dass das mit verschlüsselten Daten trainierte Modell zwar schlechter ist als die vorherig trainierten Modelle aber trotzdem in einem akzeptablen Bereich liegt.<br />
Diese Ausführung der nicht interaktiven Variante ist aber nicht die einzig mögliche. Dies liegt daran, dass homomorph verschlüsselte Objekte (wenn dies implementiert ist in der Bibliothek) auch teilweise eine Möglichkeit besitzen das Rauschen zu verringern, um wieder mehr Operationen durchführen zu können.

Austausch mit dem Datenbesitzer
---
Die erste Variante leidet unter den begrenzenten Operationen auf den verschlüsselten Gewichten. Vor allem wird dabei die Genauigkeit zugunsten der Laufzeit geopfert. In der zweiten Variante werden die Gewichte nach jeder Veränderung an den Datenbesitzer geschickt, damit diese entschlüsselt und neu verschlüsselt werden können. Damit ist eine unbegrenzte Anzahl an Operationen möglich und es kann fast wie auf unverschlüsselten Daten trainiert werden.

In [8]:
import tenseal as ts

poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 40]
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

seal_x_train = encryptX(X_train,ctx_training)
seal_y_train = encryptY(y_train,ctx_training)

Während dem Training wird also nach jedem Epoch die Gewichte dem Datenbesitzer zugesendet. Dieser ent- und verschlüsselt diese wieder mit den gleichen Parameter wie am Anfang des Trainings. Dieser Vorgang wird mit dem Aufruf der folgenden Methode simuliert.

In [9]:
def send_receive_weights(e_weights):
	weights = [item.decrypt() for item in e_weights]
	return np.array([ts.ckks_vector(ctx_training, label) for label in weights])

Der große Unterschied zum unverschlüsselten Training ist dabei, dass wir nicht aus der Ableitung der Gewichte ablesen können wie weit die Funktion noch optimiert werden muss. Dies liegt daran, da wir davon ausgehen, dass die Person, welche das Training ausführt, die entschlüsselten Gewichte erst zugesendet bekommt sobald das Training abgeschlossen ist.

In [10]:
def fit(x, y, epochs, learning_rate=0.00001):
	regression = np.c_[x, np.ones(len(x))]

	weights = np.ones(regression.shape[1])
	e_weights = np.array(encryptY(weights, ctx_training))

	for _ in range(epochs):
		y_pred = regression @ e_weights.T
		dw = regression.T @ (y - y_pred)
		e_weights = e_weights.T + (learning_rate * dw)
		e_weights = send_receive_weights(e_weights)

	return [item.decrypt() for item in e_weights]

In [11]:
e_weights = fit(seal_x_train, seal_y_train, 200, 0.0001)
e_weights = [e_weight[0] for e_weight in e_weights]

In [12]:
index = randint(0, len(X_test)-1)
print(f"""
Eingabe: {X_test[index]}
Wahres Ergebnis: {y_test[index]}
Ergebnis aus dem verschlüsselt trainierten Modell: {predict(e_weights, X_test[index])}
Ergebnis aus dem nicht verschlüsselt trainierten Modell: {predict(weights, X_test[index])}
""")


Eingabe: [-1.74885626  0.32841405 -1.39706395 -1.3154443 ]
Wahres Ergebnis: 0
Ergebnis aus dem verschlüsselt trainierten Modell: -0.009077952975548143
Ergebnis aus dem nicht verschlüsselt trainierten Modell: -0.002625144189892792



In [13]:
result = predict(e_weights, X_test)
meanError(y_test, result)

Mean Absolute Error: 0.1681699951761388
Mean Squared Error: 0.06763102721143464
Mean Root Squared Error: 0.26005966086926025


Wie zu sehen ist an den Ergebnissen hat das mit Hilfe der anderen Variante trainierte Modell eine höhere Genauigkeit. Dies wäre also ein Kompromiss zwischen der Genauigkeit und der Laufzeit, da das immer wieder aufs neue verschlüsseln der Gewichte sehr aufwändig ist.<br />
Als weitere Interaktive Version wäre es noch möglich das jeweilige Ergebnis eines Forward Schrittes zu entschlüsseln und damit das Netz zu trainieren. Dies würde die Laufzeit enorm verringern und würde die Eingabedaten nicht preisgeben, weil diese noch verschlüsselt sind und nicht für das eigentliche Training benötigt werden. Ob mit dieser Variante der Datenschutz komplett geboten ist ist fraglich.